**X-ray Image Analysis using DenseNet121**
This notebook demonstrates:
1. Preprocessing an X-ray image for feature extraction.
2. Using a **pre-trained DenseNet121 model** to extract features.
3. Generating diagnostic predictions (Normal, Pneumonia, COVID-19) using a simple text generator.

The pipeline includes:
- Loading and preprocessing an X-ray image.
- Extracting features using DenseNet121.
- Mapping the features to diagnostic labels using a simple classifier.


# Install required libraries
pip install torch torchvision pillow


## **1. Image Preprocessing**
This step involves:
- Loading the X-ray image.
- Resizing, normalizing, and converting it to a tensor for processing.

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load the pre-trained DenseNet121 model
model = models.densenet121(pretrained=True)
model.eval()  # Set to evaluation mode

# Define transformations for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model's input size
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the X-ray image
from google.colab import files
uploaded = files.upload()  # Upload the image
image_path = list(uploaded.keys())[0]  # Get the uploaded image file name
image = Image.open(image_path).convert("RGB")  # Ensure it's RGB
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension


Saving N x -ray.jpeg to N x -ray (2).jpeg


## **2. Feature Extraction Using DenseNet121**
This step involves extracting features from the image using the DenseNet121 model. Global Average Pooling is applied to reduce the size of the feature map.


In [ ]:
# Extract features using DenseNet121
with torch.no_grad():
    features = model.features(input_tensor)  # Extract feature map
    pooled_features = torch.mean(features, dim=[2, 3])  # Global Average Pooling
    print("Extracted Features Shape:", pooled_features.shape)


Extracted Features Shape: torch.Size([1, 1024])


## **3. Diagnostic Prediction**
Using a simple text generator to classify the extracted features into one of the diagnostic categories:
- Normal
- Pneumonia
- COVID-19


In [ ]:
import torch.nn as nn

# Define a simple classifier
class SimpleTextGenerator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleTextGenerator, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)  # Map features to categories

    def forward(self, x):
        return self.fc(x)

# Initialize the text generator model
text_generator = SimpleTextGenerator(input_dim=1024, output_dim=3)

# Pass the extracted features through the classifier
generated_output = text_generator(pooled_features)

# Apply softmax to get probabilities
softmax = nn.Softmax(dim=1)
probabilities = softmax(generated_output)

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=1)
print(f"Predicted Class (1=Normal, 0=Pneumonia, 2=COVID-19): {predicted_class.item()}")

# Map the predicted class to descriptive text
class_to_text = {
    0: "This X-ray is normal.",
    1: "This X-ray shows signs of pneumonia.",
    2: "This X-ray shows signs of COVID-19."
}
generated_text = class_to_text[predicted_class.item()]
print(f"Generated Text: {generated_text}")


Predicted Class (1=Normal, 0=Pneumonia, 2=COVID-19): 0
Generated Text: This X-ray is normal.


## **4. Results**
The diagnostic result will be displayed as a text description, such as:
- *"This X-ray is normal."*
- *"This X-ray shows signs of pneumonia."*
- *"This X-ray shows signs of COVID-19."*
